In [ ]:
from IPython.display import HTML
from matplotlib_inline.backend_inline import set_matplotlib_formats

# Use inline backend so CSS scaling works on PNG/SVG outputs
set_matplotlib_formats('png')

HTML("""
<style>
/* =======================
   Matplotlib figure sizing
   ======================= */

/* Images (PNG/retina) */
.jp-RenderedImage img,
.jp-OutputArea-output img {
  width: 100% !important;
  height: auto !important;
  max-width: 100% !important;
  display: block;
  box-sizing: border-box;
}

/* SVG outputs */
.jp-RenderedSVG svg,
.jp-OutputArea-output svg {
  width: 100% !important;
  height: auto !important;
}

/* ipympl / canvas-backed figures (in case %matplotlib widget is used) */
.jupyter-matplotlib canvas.mpl-canvas,
.jp-OutputArea-output canvas.mpl-canvas,
.jp-OutputArea-output canvas {
  width: 100% !important;
  height: auto !important;
  max-width: 100% !important;
}

/* Reset container widths */
.jp-OutputArea-output .matplotlib-figure,
.jp-RenderedHTMLCommon .matplotlib-figure,
.jp-OutputArea-output .output_png {
  width: 100% !important;
}

/* =======================
   ipywidgets styling
   ======================= */

/* Make slider labels bold */
.widget-label {
    font-weight: bold;
}

/* Style ipywidgets buttons */
.jp-OutputArea .widget-button,
button.widget-button {
    font-weight: bold;
    background-color: #1976d2;  /* blue background */
    color: white;               /* white text */
    border: none;
    border-radius: 6px;
    padding: 6px 12px;
    font-size: 14px;
    cursor: pointer;
}

/* Button hover effect */
.jp-OutputArea .widget-button:hover,
button.widget-button:hover {
    background-color: #1259a4;  /* darker blue */
}
</style>
""")

# Moving Targets in SAR Images 🚗  

In **SAR imaging**, the **azimuth location** of a target is determined by the **phase changes** of its echoes as the radar moves along the aperture.  

In simplified terms, SAR processing does the following:  
👉 Pick a pixel location on the ground.  
👉 For each radar position along the synthetic aperture, compute the **range (and phase)** of the echo from that location.  
👉 Align the measured echoes according to these ranges and phases, then **sum them coherently**. ✨  

This works beautifully as long as two key assumptions hold true:  
1. We know the trajectory of our radar platform (the orbit). 🛰️  
2. The target on the ground is **stationary**. 🪨  

But what if that second assumption is violated? 🤔 Imagine a **car driving down a road** in the illuminated scene. 🚘 Now the **range and phase** of the echo from the car change in a way that **does not match** the fixed-location assumption used in SAR processing when we steer the beam of our virtual antenna array.  

The result? The moving target can be **misfocused and displaced** in the final SAR image. This leads to some very interesting—and sometimes surprising—effects. 🌪️  

Let’s dive in and explore what moving targets look like in SAR imagery! 🔍

## 🚗➡️ Target Moving Toward or Away from the Radar  

So far, we’ve learned that SAR works a bit like a **giant phased array**:  
- Every position along the flight path is like a new “antenna element.”  
- By comparing the **delay and phase patterns** of echoes across this synthetic array, the processor figures out where in azimuth the echo came from.  
- Just like our **buoy array** on the lake: if the wave comes straight ahead, all buoys rock in sync; if the wave comes at an angle, each buoy gets nudged at slightly different times → giving us phase differences that reveal the azimuth of the target. 🛟🌊  

👉 The key is: the **pattern of phase differences** tells the radar *where to place the target* in azimuth.

---

### Now bring in the moving car 🚗💨  

Imagine a car driving **toward the radar** (along the ground range axis).  

- For a stationary target, the only phase differences come from its **azimuth position** (like a wave hitting the buoys from some angle).  
- But if the target is moving in range while we collect data, its **distance keeps changing** between the first and last “buoys.”  
- That motion adds an **extra tilt** to the phase pattern. *In the buoy picture, it’s as if the wave is tilted more steeply than expected — so the processor thinks it came from a different angle.* 🌊   

The processor can’t tell where that tilt came from — it only sees the **total phase slope** across the aperture.  

👉 And here’s the key: that slope **looks identical** to what a stationary target at another azimuth would produce. In effect, the moving car is **impersonating** a target from somewhere else along the azimuth axis!  

- Faster velocity → bigger extra slope → larger azimuth displacement.  
- Slower velocity → smaller extra slope → smaller displacement.   

---

### Flip the case ↩️  

If the car moves in the **opposite range direction** (toward vs. away), the extra slope just flips sign.  
👉 That means the azimuth shift happens the **other way**.  

- **Motion toward the radar** → the range shrinks faster → the phase pattern looks like a target the radar would see *later* along the aperture.  
  👉 The target is shifted **away from the early azimuth positions** (toward later azimuth).  
- **Motion away from the radar** → the range shrinks more slowly → the phase pattern looks like a target the radar would see *earlier* along the aperture.  
  👉 The target is shifted **toward the early azimuth positions**.  

The key is: the *magnitude* of the shift depends on the speed, and the *direction* depends on whether the target is moving toward or away.  

---

### 🌀 Peculiar Consequences  
This effect leads to some strange-looking SAR images:  
- Cars may appear as if they’re **driving beside the road** instead of on it. 🚗🛣️  
- Even more bizarrely, **trains can appear displaced off their tracks**! 🚂  

But once we look carefully at the physics, the mystery clears up. SAR forms its sharp images by **combining measurements made at different times along the aperture**. Since the processing assumes a **stationary scene**, any motion breaks that assumption and affects the placement of the echoes.

### 🎮 Simulation Time  

Let’s bring this concept to life with a little interactive geometry show!  

- 🚀 The satellite cruises along its flight path high above.  
- 📡 Its radar beam footprint illuminates the ground below in spotlight mode.  
- 🚗 A **moving target** drives inside that illuminated patch.  

👉 Play with the sliders to set the **speed** and **direction** of the target.  

On the **right-hand plot**, you’ll see three curves of slant-range distance vs. time:  
- **Moving Target** — the real path of our car as it zips along.  
- **Matched Patch** — the stationary spot in the scene the processor *thinks* matches it → this is why the car shows up displaced in azimuth.  
- **True-x, fixed-y₀** — the range history at the car’s *true azimuth location* but fixed range row; this helps visualize why the curves don’t line up.  

✨ The gap between these curves is the key to understanding why moving objects appear in the *wrong place* in SAR images.


In [ ]:
def build_mti_voila_app():
    # Miniature Voilá-ready SAR moving-target demo (ALL METERS).
    # - Fixed 5 s collection
    # - Left: 3D spotlight geometry
    # - Right: Slant Range R(t) for:
    #     • Moving target
    #     • Matched stationary patch at (x_match, y0)
    #     • True azimuth x_target but FIXED range y0  ← NEW: y does not follow the target
    #
    # Coordinates: meters; Velocities: m/s.
    import numpy as np
    import matplotlib.pyplot as plt
    from mpl_toolkits.mplot3d import Axes3D  # noqa: F401
    from mpl_toolkits.mplot3d.art3d import Poly3DCollection
    from ipywidgets import FloatSlider, IntSlider, Play, jslink, Button, Layout, Output, Label, Dropdown, HBox, VBox
    from IPython.display import display, clear_output

    # ---------- Constants (no UI) ----------
    SATELLITE_VELOCITY_MS = 50       # m/s (tiny for demo)
    ALTITUDE_M            = 1500.0   # m
    INCIDENCE_DEG         = 45.0     # deg
    BEAM_HALF_ANGLE_DEG   = 5.0      # deg
    FPS                   = 12       # frames per second
    COLLECTION_TIME_S     = 5.0      # fixed collection time

    # ---------- Controls ----------
    target_speed = FloatSlider(
        value=2.0, min=0.0, max=10.0, step=1.0,
        description="Speed (m/s)", continuous_update=False
    )
    direction = Dropdown(
        options=[("Toward radar (+y)", +1), ("Away from radar (−y)", -1)],
        value=+1, description="Direction"
    )

    # Timeline controls
    frames = int(COLLECTION_TIME_S * FPS)
    play = Play(interval=80, value=0, min=0, max=frames-1, step=1)
    frame_slider = IntSlider(value=0, min=0, max=frames-1, step=1, description="Frame", continuous_update=True)
    jslink((play, 'value'), (frame_slider, 'value'))

    reset_btn = Button(description="Reset", layout=Layout(width='90px'))
    recenter_btn = Button(description="Recenter", layout=Layout(width='110px'))

    out = Output()

    # ---------- State ----------
    state = dict(
        x_positions=None, y_positions=None, z_positions=None,
        artists={}, fig=None, theta=np.linspace(0, 2*np.pi, 160),
        frames=frames, t=None, match_x=None, dx_est=None,
        Rm=None, Rs=None, Rxf=None  # Rxf = range to (x_target, y0) with FIXED y
    )

    ground_point = (0.0, 0.0, 0.0)

    # ---------- Helpers ----------
    def _compute_geometry():
        aperture_length = SATELLITE_VELOCITY_MS * COLLECTION_TIME_S
        xs = np.linspace(-aperture_length/2.0, aperture_length/2.0, state['frames'])

        # Scene center (ground point) at incidence angle
        y_scene = ALTITUDE_M * np.tan(np.radians(INCIDENCE_DEG))
        ys = np.full_like(xs, y_scene)
        zs = np.full_like(xs, ALTITUDE_M)

        # Time vector (full 5 s)
        t = np.linspace(-COLLECTION_TIME_S/2.0, COLLECTION_TIME_S/2.0, state['frames'])

        state['x_positions'], state['y_positions'], state['z_positions'] = xs, ys, zs
        state['t'] = t

        # ---------- AXIS LIMITS ----------
        gp_x, gp_y, gp_z = ground_point
        rngs = np.sqrt((xs - gp_x)**2 + (ys - gp_y)**2 + (zs - gp_z)**2)
        hw_arr = np.tan(np.radians(BEAM_HALF_ANGLE_DEG)) * rngs
        max_hw = float(hw_arr.max())  # max azimuth half-width of footprint over the dwell

        half_span_path = 0.5 * (xs.max() - xs.min())
        half_span = max(max_hw, half_span_path)
        pad = 0.5 * max(1.0, half_span)   # 5% padding
        xlim = (-half_span - pad, half_span + pad)

        ylim = (-0.2 * (abs(y_scene) + 1.0), y_scene + abs(y_scene) * 0.25 + 5.0)
        zlim = (0.0, ALTITUDE_M * 1.1 + 5.0)

        return xlim, ylim, zlim, y_scene

    def _slant_range(xp, yp, zp, xg, yg):
        return np.sqrt((xp - xg)**2 + (yp - yg)**2 + (zp - 0.0)**2)

    def _estimate_static_match_x(x_target, y0, v_rg_ms, xs, ys, zs, t):
        # Match by minimizing mean-removed shape error; plot absolute ranges.
        y_mov = y0 + v_rg_ms * t
        Rm = _slant_range(xs, ys, zs, x_target, y_mov)

        pad = 200.0
        x_candidates = np.linspace(xs.min() - pad, xs.max() + pad, 401)
        best_x = None
        best_err = np.inf
        for x_s in x_candidates:
            Rs = _slant_range(xs, ys, zs, x_s, y0)
            err = np.mean(((Rm - Rm.mean()) - (Rs - Rs.mean()))**2)
            if err < best_err:
                best_err = err
                best_x = x_s

        dx = best_x - x_target
        return best_x, dx

    # ---------- Plot init ----------
    def _init_plot():
        xlim, ylim, zlim, _ = _compute_geometry()

        # 1x2 layout: left = 3D scene, right = absolute range plot
        fig = plt.figure(figsize=(13.0, 6.0))
        gs = fig.add_gridspec(1, 2, width_ratios=[3, 2], wspace=0.35)
        ax3d = fig.add_subplot(gs[0, 0], projection='3d')
        axr  = fig.add_subplot(gs[0, 1])

        # --- Left: 3D scene ---
        ax3d.set_title("Spotlight Geometry")
        ax3d.set_xlabel("X (m) — azimuth")
        ax3d.set_ylabel("Y (m) — ground range")
        ax3d.set_zlabel("Z (m) — altitude")
        ax3d.set_xlim(*xlim); ax3d.set_ylim(*ylim); ax3d.set_zlim(*zlim)

        flight_path = ax3d.plot(state['x_positions'], state['y_positions'], state['z_positions'],
                                lw=1.0, alpha=0.7, label="Flight path")[0]

        gp_dot = ax3d.plot([ground_point[0]], [ground_point[1]], [ground_point[2]],
                           marker='o', markersize=5, color='k', label="Scene Center")[0]

        los_line   = ax3d.plot([], [], [], linestyle='--', lw=1.3, label="Beam Center")[0]
        beam_left  = ax3d.plot([], [], [], linestyle='--', lw=1.0, label="Beam Edge")[0]
        beam_right = ax3d.plot([], [], [], linestyle='--', lw=1.0)[0]

        aperture_line, = ax3d.plot([], [], [], lw=3.0, label="Aperture accrued")

        footprint = Poly3DCollection([[(0,0,0)]], alpha=0.35)
        ax3d.add_collection3d(footprint)

        sat_marker = ax3d.scatter(
            [state['x_positions'][0]], [state['y_positions'][0]], [state['z_positions'][0]],
            marker='^', s=120, edgecolors='k', linewidths=0.6, label='Satellite'
        )

        target_marker = ax3d.scatter([-40.0], [0.0], [0.0],
                                     marker='s', s=60, edgecolors='k', linewidths=0.6, label='Moving target')

        match_marker = ax3d.scatter([0.0], [0.0], [0.0],
                                    marker='o', s=70, linewidths=0.6, label='Matched stationary patch')

        ax3d.legend(loc='upper left', fontsize=9)

        # --- Right: absolute range plot ---
        axr.set_title("Slant Range R(t): Moving vs Matched vs True‑x (fixed‑y₀)")
        axr.set_xlabel("Time (s)")
        axr.set_ylabel("Slant Range (m)")
        axr.grid(True, alpha=0.4)
        axr.set_xlim(-COLLECTION_TIME_S/2.0, COLLECTION_TIME_S/2.0)

        r_mov_line,   = axr.plot([], [], lw=2, label="Moving target R(t)")
        r_stat_line,  = axr.plot([], [], lw=2, linestyle='--', label="Matched patch (x_match, y0)")
        r_truex_line, = axr.plot([], [], lw=2, linestyle='-.', label="True‑x, fixed‑y₀ (x_target, y0)")  # NEW
        cursor_line    = axr.plot([], [], lw=1, alpha=0.7)[0]
        axr.legend(loc="upper right", fontsize=9)

        state['fig'] = fig
        state['artists'] = dict(
            ax3d=ax3d, axr=axr,
            flight_path=flight_path, gp_dot=gp_dot,
            los_line=los_line, beam_left=beam_left, beam_right=beam_right,
            aperture_line=aperture_line, footprint=footprint, gp=ground_point,
            sat_marker=sat_marker, target_marker=target_marker, match_marker=match_marker,
            r_mov_line=r_mov_line, r_stat_line=r_stat_line, r_truex_line=r_truex_line,
            cursor_line=cursor_line
        )

    # ---------- Render (Voilá-safe) ----------
    def _render():
        with out:
            clear_output(wait=True)
            display(state['fig'])
            plt.close(state['fig'])

    # ---------- Frame update ----------
    def _update_frame(k):
        if state['x_positions'] is None:
            return

        xs, ys, zs, t = state['x_positions'], state['y_positions'], state['z_positions'], state['t']
        xk, yk, zk = xs[k], ys[k], zs[k]
        A = state['artists']; gp = A['gp']

        # --- 3D: beam center LOS ---
        A['los_line'].set_data([xk, gp[0]], [yk, gp[1]])
        A['los_line'].set_3d_properties([zk, gp[2]])

        # --- 3D: beam edges ---
        rng = np.sqrt((xk-gp[0])**2 + (yk-gp[1])**2 + (zk-gp[2])**2)
        hw = np.tan(np.radians(BEAM_HALF_ANGLE_DEG)) * rng
        A['beam_left'].set_data([xk, gp[0] - hw], [yk, gp[1]])
        A['beam_left'].set_3d_properties([zk, gp[2]])
        A['beam_right'].set_data([xk, gp[0] + hw], [yk, gp[1]])
        A['beam_right'].set_3d_properties([zk, gp[2]])

        # --- 3D: footprint ---
        cosi = max(np.cos(np.radians(INCIDENCE_DEG)), 1e-6)
        a_az = hw
        a_rg = hw / (cosi)
        th = state['theta']
        xc = gp[0] + a_az * np.cos(th)
        yc = gp[1] + a_rg * np.sin(th)
        zc = np.zeros_like(th)
        A['footprint'].set_verts([list(zip(xc, yc, zc))])

        # --- 3D: aperture so far ---
        A['aperture_line'].set_data(xs[:k+1], ys[:k+1])
        A['aperture_line'].set_3d_properties(zs[:k+1])

        # --- 3D: satellite marker ---
        A['sat_marker']._offsets3d = ([xk], [yk], [zk])

        # --- Moving target position (ground) ---
        v_rg_ms = target_speed.value * direction.value
        x_target = -40.0
        y0 = 0.0
        y_now = y0 + v_rg_ms * t[k]
        A['target_marker']._offsets3d = ([x_target], [y_now], [0.0])

        # --- Compute matched x and all absolute range histories (once per rebuild) ---
        if state['match_x'] is None:
            x_s, dx = _estimate_static_match_x(x_target, y0, v_rg_ms, xs, ys, zs, t)
            state['match_x'] = x_s
            state['dx_est'] = dx

            y_mov = y0 + v_rg_ms * t
            Rm  = _slant_range(xs, ys, zs, x_target, y_mov)      # moving target (x_target, y(t))
            Rs  = _slant_range(xs, ys, zs, state['match_x'], y0) # matched patch (x_match, y0)
            Rxf = _slant_range(xs, ys, zs, x_target, y0)         # NEW: true x but FIXED y0

            state['Rm'] = Rm
            state['Rs'] = Rs
            state['Rxf'] = Rxf

        # --- 3D: matched patch marker ---
        A['match_marker']._offsets3d = ([state['match_x']], [y0], [0.0])

        # --- RIGHT plot: absolute ranges & time cursor ---
        if state['Rm'] is not None and state['Rs'] is not None:
            A['r_mov_line'].set_data(t, state['Rm'])
            A['r_stat_line'].set_data(t, state['Rs'])
            if state['Rxf'] is not None:
                A['r_truex_line'].set_data(t, state['Rxf'])

            # Rescale y based on all traces
            vals_min = [state['Rm'].min(), state['Rs'].min()]
            vals_max = [state['Rm'].max(), state['Rs'].max()]
            if state['Rxf'] is not None:
                vals_min.append(state['Rxf'].min())
                vals_max.append(state['Rxf'].max())
            ymin = float(min(vals_min)); ymax = float(max(vals_max))
            pad = 0.05 * max(1.0, (ymax - ymin))
            A['axr'].set_ylim(ymin - pad, ymax + pad)

            # Time cursor
            yl0, yl1 = A['axr'].get_ylim()
            A['cursor_line'].set_data([t[k], t[k]], [yl0, yl1])

        _render()

    # ---------- Rebuild & callbacks ----------
    def _rebuild_scene(*_):
        _compute_geometry()
        play.min = 0; play.max = state['frames'] - 1
        frame_slider.min = 0; frame_slider.max = state['frames'] - 1
        frame_slider.value = 0

        # Reset caches
        state['match_x'] = None
        state['dx_est'] = None
        state['Rm'] = None
        state['Rs'] = None
        state['Rxf'] = None  # reset

        _init_plot()
        _update_frame(0)

    def _on_reset(_):
        frame_slider.value = 0
        _update_frame(0)

    def _on_recenter(_):
        xlim, ylim, zlim, _ = _compute_geometry()
        ax3d = state['artists']['ax3d']
        ax3d.set_xlim(*xlim); ax3d.set_ylim(*ylim); ax3d.set_zlim(*zlim)
        _render()

    def _on_speed_or_dir_change(_):
        _rebuild_scene()

    frame_slider.observe(lambda ch: _update_frame(ch['new']), names='value')
    reset_btn.on_click(_on_reset)
    recenter_btn.on_click(_on_recenter)
    target_speed.observe(_on_speed_or_dir_change, names='value')
    direction.observe(_on_speed_or_dir_change, names='value')

    # Build once
    _rebuild_scene()

    # ---------- Layout ----------
    header = Label(
        "Moving Target Demo (meters): left = 3D geometry, right = Slant Range R(t)\n"
        "• Fixed 5 s collection; adjust target speed & direction.\n"
        "Curves: Moving, Matched (x_match,y0), True‑x fixed‑y₀ (x_target,y0)."
    )
    controls = HBox([target_speed, direction, reset_btn, recenter_btn])
    timeline = HBox([play, frame_slider])

    return VBox([header, controls, timeline, out])


# Build & display
mti_app = build_mti_voila_app()
display(mti_app)

## 🚙 Targets Moving Along the Aperture or Accelerating 🚀

So far, we looked at targets moving steadily **towards or away** from the radar (range direction). That gave us a **constant extra phase slope** → just a clean shift in azimuth.  

Now let’s imagine a different case:  
👉 the target is moving **along the synthetic aperture itself** — i.e. sideways, in the **azimuth direction**.

---

### 🔍 Why does this matter?

In SAR, what really counts is **how the slant range changes from pulse to pulse**. That’s because the slant range tells us how far the echo wave has traveled — which directly sets its **delay and phase**.  

- **Range velocity (constant)** → the slant range changes at a steady rate.  
  Every pulse sees the *same* extra distance increment.  
  ➡️ That’s why it gave us a **simple shift** in azimuth.  

- **Azimuth velocity** → the slant range change is *not steady*.  
  From one pulse to the next, the distance increments vary, because the target is sliding sideways relative to the radar.  
  That means the “best-matching azimuth pixel” keeps jumping around.  

---

### 🎨 What does it look like in the image?

Instead of all the energy stacking up neatly in one pixel, it gets **spread out across azimuth**:  

- Faster sideways motion → **more smear**.  
- In strong cases, the target becomes a **streak** stretched over many azimuth cells.  
- Think of it as a **paintbrush stroke** across the image, instead of a sharp dot. 🎨  

---

### ⚡️ What about acceleration?

Acceleration breaks the “steady slope” rule as well:  

- **Constant range velocity** → steady phase slope → shift.  
- **Acceleration (in range)** → phase slope keeps changing → smear.  

So azimuth velocity *and* range acceleration both lead to **streaking instead of shifting**.  

---

### 🌍 The general case

In reality, targets can move in **any direction** with **any motion pattern**:  
- **Range velocity** → clean displacement.  
- **Azimuth velocity or range acceleration** → smearing.  
- Mix them together → displacement + streaking, which is why moving-target detection in SAR is such a tricky business. 🚙💡  

## 🚗💨 Displacement vs. Smearing in SAR Images  

To make the effects of motion more tangible, below is a simple **SAR-like image simulation**:  

- The scene shows a **crossroad**:  
  - 🛣️ **Dark asphalt roads** (low backscatter)  
  - ✨ **Bright fences** along the road edges  
  - 🌾 **Speckly fields** in the background  

- A small **rectangular car** is placed at the center, and it's driving through the crossroads. 🚦 

👉 Use the controls to experiment:
- **Motion along Range (vertical):** the car gets **displaced in azimuth**.  
  The faster it goes, the further it shifts sideways — at maximum speed, it jumps all the way to the edge.  
- **Motion along Azimuth (horizontal):** the car gets **smeared** along azimuth.  
  The faster it goes, the longer the streak — at maximum speed, it stretches across half the image.  

This illustrates the two key motion effects in SAR:  
- 📍 **Displacement** when the velocity produces a constant range change per pulse.  
- 🌪️ **Smearing** when the range change varies pulse by pulse.  

In [ ]:
def build_sar_motion_demo():
    """
    Voilá-ready interactive demo with signed velocities:
    - Range velocity (±) -> azimuth displacement (left/right)
    - Azimuth velocity (±) -> directional azimuth smear (left/right)
    """
    import io
    import numpy as np
    import matplotlib.pyplot as plt
    from ipywidgets import VBox, HBox, FloatSlider, Label, Image as WImage, Layout, HTML
    from IPython.display import display

    # -------------------- Scene constants --------------------
    H, W = 500, 500
    center_y, center_x = H // 2, W // 2

    road_w, road_level_dark = 18, 0.06
    fence_level, fence_offset, fence_thick = 0.95, 2, 2
    max_velocity = 30.0

    car_h, car_w = 14, 10
    car_level    = 0.98

    # -------------------- Widgets --------------------
    v_range = FloatSlider(value=0.0, min=-max_velocity, max=max_velocity, step=0.1,
                          description="Range vel. (m/s)", continuous_update=False,
                          readout=True, layout=Layout(width="360px"))
    v_range.style.description_width = 'initial'
    v_az    = FloatSlider(value=0.0, min=-max_velocity, max=max_velocity, step=0.1,
                          description="Azimuth vel. (m/s)", continuous_update=False,
                          readout=True, layout=Layout(width="360px"))
    v_az.style.description_width = 'initial'
    info   = Label("Range → left/right shift; Azimuth → left/right smear (signed).")
    out_img = WImage(format='png', layout=Layout(width="560px", height="560px"))
    readout = HTML()
    header  = HTML("<b>SAR Motion Demo: signed displacement + directional smear</b>")

    # -------------------- Helpers --------------------
    def _box_blur_1d(arr, k):
        if k <= 1: return arr.copy()
        pad = k // 2
        arr_pad = np.pad(arr, ((0,0),(pad,pad)), mode='edge')
        kernel = np.ones(k, dtype=np.float32) / k
        return np.apply_along_axis(lambda r: np.convolve(r, kernel, mode='valid'), axis=1, arr=arr_pad)

    def _smooth2d(arr, kx, ky):
        tmp = _box_blur_1d(arr, kx)
        tmp = _box_blur_1d(tmp.T, ky).T
        return tmp

    # -------------------- Background (CACHED) --------------------
    def make_background_cached():
        rng = np.random.default_rng(42)
        base = rng.random((H, W), dtype=np.float32)
        base = _smooth2d(base, kx=31, ky=31)
        base = (base - base.min()) / max(1e-6, (base.max() - base.min()))
        base = 0.28 + 0.30 * base
        sigma   = 1.0 / np.sqrt(np.pi / 2.0)
        speckle = rng.rayleigh(scale=sigma, size=(H, W)).astype(np.float32)
        img = np.clip(base * speckle, 0.0, 1.0)

        x0, x1 = center_x - road_w//2, center_x + road_w//2
        y0, y1 = center_y - road_w//2, center_y + road_w//2
        img[:, x0:x1] = np.minimum(img[:, x0:x1], road_level_dark)
        img[y0:y1, :] = np.minimum(img[y0:y1, :], road_level_dark)

        vf_left0  = max(0, x0 - fence_offset - fence_thick//2)
        vf_left1  = min(W, vf_left0 + fence_thick)
        vf_right0 = max(0, x1 + fence_offset - fence_thick//2)
        vf_right1 = min(W, vf_right0 + fence_thick)
        img[:, vf_left0:vf_left1]   = np.maximum(img[:, vf_left0:vf_left1],   fence_level)
        img[:, vf_right0:vf_right1] = np.maximum(img[:, vf_right0:vf_right1], fence_level)

        hf_top0 = max(0, y0 - fence_offset - fence_thick//2)
        hf_top1 = min(H, hf_top0 + fence_thick)
        hf_bot0 = max(0, y1 + fence_offset - fence_thick//2)
        hf_bot1 = min(H, hf_bot0 + fence_thick)
        img[hf_top0:hf_top1, :] = np.maximum(img[hf_top0:hf_top1, :], fence_level)
        img[hf_bot0:hf_bot1, :] = np.maximum(img[hf_bot0:hf_bot1, :], fence_level)
        return img

    BG = make_background_cached()

    def paint_rect(img, cy, cx, hh, hw, level):
        Hh, Ww = img.shape
        y0 = max(0, int(round(cy - hh)))
        y1 = min(Hh, int(round(cy + hh)))
        x0 = max(0, int(round(cx - hw)))
        x1 = min(Ww, int(round(cx + hw)))
        if y0 < y1 and x0 < x1:
            img[y0:y1, x0:x1] = level
        return y0, y1, x0, x1

    # -------------------- Directional smear kernel --------------------
    def make_directional_kernel(klen, sign):
        """
        klen: odd length >= 3
        sign: +1 for rightward smear, -1 for leftward smear
        Nonzeros lie on the pivot side including the pivot.
        """
        klen = int(klen)
        if klen % 2 == 0:
            klen -= 1
        klen = max(klen, 3)
        pivot = klen // 2  # center index
        k = np.zeros(klen, dtype=np.float32)
        if sign >= 0:
            k[pivot:] = 1.0
        else:
            k[:pivot+1] = 1.0
        # Normalize area; we'll still re-normalize peak later to car_level
        s = k.sum()
        if s > 0:
            k /= s
        return k, klen

    # -------------------- Combined renderer --------------------
    def render_combined(v_range_val, v_az_val):
        # 1) Range displacement (signed)
        max_dx = (W - 1) - center_x
        dx = int(round((v_range_val / max_velocity) * max_dx))
        cx = int(np.clip(center_x + dx, 0, W-1))

        car = np.zeros_like(BG, dtype=np.float32)
        y0, y1, x0, x1 = paint_rect(car, center_y, cx, car_h // 2, car_w // 2, car_level)

        # 2) Azimuth smear (signed & directional) on car rows only
        vabs = abs(v_az_val)
        if vabs > 0.0:
            desired_len = int(round((vabs / max_velocity) * (W // 2)))
            if desired_len > 1:
                kernel, klen = make_directional_kernel(desired_len, np.sign(v_az_val))
                smeared = np.zeros_like(car)
                for r in range(y0, y1):
                    smeared[r, :] = np.convolve(car[r, :], kernel, mode='same')
                # Normalize to keep peak brightness ~car_level
                maxv = float(smeared.max())
                if maxv > 1e-12:
                    smeared *= (car_level / maxv)
                car_layer = smeared
                smear_px = klen
            else:
                car_layer = car
                smear_px = 1
        else:
            car_layer = car
            smear_px = 1

        # 3) Composite (brighter wins)
        img = np.clip(np.maximum(BG, car_layer), 0.0, 1.0)
        return img, dx, smear_px

    # -------------------- PNG rendering --------------------
    DPI = 110
    def _render_png(array2d, title):
        fig, ax = plt.subplots(figsize=(5.6, 5.6), dpi=DPI)
        ax.imshow(array2d, cmap="gray", vmin=0.0, vmax=1.0, origin='upper', interpolation="nearest")
        ax.set_xlabel("Azimuth (pixels)")
        ax.set_ylabel("Range (pixels)")
        ax.set_title(title)
        fig.tight_layout()
        buf = io.BytesIO()
        fig.savefig(buf, format='png', dpi=DPI)
        plt.close(fig)
        buf.seek(0)
        return buf.getvalue()

    # -------------------- Render --------------------
    def render(_=None):
        img, dx, smear_px = render_combined(v_range.value, v_az.value)
        title = (f"Combined motion | v_range={v_range.value:.2f} m/s, "
                 f"v_az={v_az.value:.2f} m/s")
        out_img.value = _render_png(img, title)
        readout.value = ("<span style='color:gray'>"
                         f"Grid: {H}×{W} px &nbsp;|&nbsp; Car: {car_w}×{car_h} px &nbsp;|&nbsp; "
                         "</span>")

    v_range.observe(render, names="value")
    v_az.observe(render, names="value")

    # Initial draw
    render()

    controls = HBox([v_range, v_az], layout=Layout(column_gap="12px"))
    return VBox([header, controls, info, readout, out_img])

# Build & display
motion_demo = build_sar_motion_demo()
display(motion_demo)